In [1]:
import gzip
import numpy as np
import opencorpora

from collections import defaultdict, Counter
from abc import ABCMeta, abstractmethod
from operator import itemgetter, attrgetter

# Opencorpora corpus

In [2]:
corpus = opencorpora.load('../annot.opcorpora.xml')

# Analyzer

In [13]:
class Node:
    def __init__(self):
        self.data = None
        self.next = {}

class Trie:
    def __init__(self):
        self._root = Node()
    
    def add(self, text, data):
        node = self._root
        
        for ch in text:
            if ch not in node.next:
                node.next[ch] = Node()
            
            node = node.next[ch]
            
        node.data = data
            
    def get(self, text):
        node = self._root
        
        for ch in text:
            if ch not in node.next:
                return None
            
            node = node.next[ch]
            
        return node.data

In [19]:
class Analyzer:
    def __init__(self, default_pos='S'):
        self._trie = Trie()
        self.default_pos = default_pos
        
    def split(self, sentence: str) -> str:
        return ''.join(ch for ch in sentence if ch not in ',.?!').split()
    
    def parse_word(self, word: str) -> str:
        counter = self._trie.get(word) or self._trie.get(word.lower())
        
        if not counter:
            lemma, pos = word.lower(), self.default_pos
        else:
            (lemma, pos), _ = counter.most_common()[0]
            
        return '{}{{{}={}}}'.format(word, lemma, pos)
    
    def parse(self, sentence: str) -> str:
        return ' '.join(self.parse_word(word) for word in self.split(sentence)) 

## ODict analyzer

In [20]:
class ODictAnalyzer(Analyzer):
    ODICT_TO_POS = {
        'вводн.': 'ADV',
        'ж': 'S',
        'жо': 'S',
        'м': 'S',
        'межд.': 'ADV',
        'мн.': 'S',
        'мо': 'S',
        'мо-жо': 'S',
        'мс-п': 'A',
        'н': 'ADV',
        'нсв': 'V',
        'п': 'A',
        'предик.': 'ADV',
        'предл.': 'PR',
        'с': 'S',
        'св': 'V',
        'св-нсв': 'V',
        'со': 'S',
        'союз': 'CONJ',
        'сравн.': 'ADV',
        'част.': 'ADV',
        'числ.': 'UNK',
        'числ.-п': 'A'
    }
    
    def __init__(self, path='../odict.csv.gz'):
        super().__init__()
        self._build(path)
        
    def _build(self, path: str):
        with gzip.open(path, mode='rt') as odict:
            for line in odict:
                lemma, t, *words = line.split(',')[:-1]
                pos = self.ODICT_TO_POS[t]
                
                for word in words:
                    counter = self._trie.get(word)
                    
                    if not counter:
                        counter = Counter()
                        self._trie.add(word, counter)
                    
                    counter.update({(lemma, pos): 1})

## Opencorpora analyzer

In [51]:
class OpenCorporaAnalyzer(Analyzer):
    OPENCORPORA_TO_POS = {
        'NOUN': 'S',
        'ADJF': 'A',
        'ADJS': 'A',
        'VERB': 'V',
        'INFN': 'V',
        'PRTF': 'V',
        'PRTS': 'V',
        'GRND': 'V',
        'ADVB': 'ADV',
        'NPRO': 'S',
        'PREP': 'PR',
        'CONJ': 'CONJ',
        'PRCL': 'ADV',
        'INTJ': 'ADV',
    }
    
    def __init__(self, corpus):
        super().__init__()
        self._build(corpus)
        
    def _build(self, corpus):
        for doc in corpus:
            for sentence in doc.sentences:
                for token in sentence.tokens:
                    lemma, t = token.parse.lemma, token.parse.grammemes[0]
                    pos = self.OPENCORPORA_TO_POS.get(t, None) or 'UNK'
                    
                    counter = self._trie.get(token.source)

                    if not counter:
                        counter = Counter()
                        self._trie.add(token.source, counter)

                    counter.update({(lemma, pos): 1})

In [89]:
class MixedAnalyzer(Analyzer):
    def __init__(self, od_trie: Trie, oc_trie: Trie):
        super().__init__()
        self._od_trie = od_trie
        self._oc_trie = oc_trie
        
    @staticmethod
    def extract(trie, word):
        counter = trie.get(word) or trie.get(word.lower())
        
        if counter:
            (lemma, pos), _ = counter.most_common()[0]
            return lemma, pos
        
        return None, None
        
    def parse_word(self, word: str) -> str:
        lemma, pos = self.extract(self._oc_trie, word)
        
        if lemma:
            od_lemma_1, _ = self.extract(self._od_trie, lemma)
            od_lemma_2, _ = self.extract(self._od_trie, word)
            lemma = (od_lemma_1 or od_lemma_2) or lemma
        else:
            lemma, pos = self.extract(self._od_trie, word)
            lemma = lemma or word.lower()
            pos = pos or self.default_pos
        
        _, oc_pos = self.extract(self._oc_trie, lemma)
        pos = oc_pos or pos
        
        return '{}{{{}={}}}'.format(word, lemma, pos)

# Test

In [22]:
oDictAnalyzer = ODictAnalyzer()

In [ ]:
openCorporaAnalyzer = OpenCorporaAnalyzer(corpus)

In [90]:
mixedAnalyzer = MixedAnalyzer(oDictAnalyzer._trie, openCorporaAnalyzer._trie)

In [92]:
dataset = [
    'Стала стабильнее экономическая и политическая обстановка, предприятия вывели из тени зарплаты сотрудников.',
    'Все Гришины одноклассники уже побывали за границей, он был чуть ли не единственным, кого не вывозили никуда дальше Красной Пахры.'
]

for sentence in dataset:
    print(mixedAnalyzer.parse(sentence))

Стала{стать=V} стабильнее{стабильный=A} экономическая{экономический=A} и{и=CONJ} политическая{политический=A} обстановка{обстановка=S} предприятия{предприятие=S} вывели{вывести=V} из{из=PR} тени{тень=S} зарплаты{зарплата=S} сотрудников{сотрудник=S}
Все{весить=A} Гришины{гришин=A} одноклассники{одноклассник=S} уже{уж=ADV} побывали{побывать=V} за{за=PR} границей{граница=S} он{он=S} был{есть=V} чуть{чуть=ADV} ли{ли=CONJ} не{не=ADV} единственным{единственный=A} кого{кто=S} не{не=ADV} вывозили{выводить=V} никуда{никуда=ADV} дальше{далёкий=A} Красной{красный=A} Пахры{пахры=S}


In [95]:
with open('dataset_37845_1-8.txt', 'r') as file:
    dataset = file.readlines()
    
for sentence in dataset:
    print(mixedAnalyzer.parse(sentence))

Однажды{однажды=ADV} пришла{прийти=V} женщина{женщина=S} с{с=PR} лицом{лицо=S} бледной{бледный=A} стеариновой{стеариновый=A} желтизны{желтизна=S} закутанная{закутать=V} как{как=CONJ} бабка{бабка=S} бессловесная{бессловесный=A} безулыбчивая{безулыбчивая=S} но{но=CONJ} глаза{глаз=S} сияли{сиять=V} ясно{ясный=A} голубо{голубо=S}
Раздел{раздел=S} Параметры{параметр=S} определяет{определять=V} предметные{предметный=A} свойства{свойство=S} изделия{изделие=S} т{так=CONJ} е{есть=V} имеющие{иметь=V} отношение{отношение=S} к{к=PR} определяемому{определять=V} конструкторскому{конструкторский=A} понятию{понятие=S} сущности{сущность=S}
Это{это=ADV} означало{означать=V} отсутствие{отсутствие=S} в{в=PR} раздатке{раздатке=S} дифференциала{дифференциал=S} и{и=CONJ} соответственно{соответственный=ADV} невозможность{невозможность=S} управлять{управлять=V} автомобилем{автомобиль=S} в{в=PR} режиме{режим=S} полного{полный=A} привода{привод=S} на{на=PR} сухом{сухой=A} асфальте{асфальт=S}
Живописные{живописны